# Cross clock domain logic

When several independent clocks are present in a design and data needs to be passed between those clock domains, a few extensions may come in handy.
For instance, an ethernet interface may run on 25 MHz master clock, whereas the CPU core is run on 54 MHz. To pass data between the DMA logic of the CPU and the Ethernet core, a packet fifo may come in handy, that uses dual port RAM to allow data streaming in an efficient manner.
Such packet FIFO designs are normally based on gray counters, plus they might need to contain logic to pass flags reliably from one domain to the other.

The `xcd` library supplies a few primitives for transferring a pulse across clock domains.

## Single pulse transfer

The unit below takes two differing input clocks with input `a` and output `b`, respectively. An input pulse of a duration of one clock period of `clka` will result in a `clkb`-synchronized output pulse. This is achieved by the `XClkDomainSynced` logic which is called like a function, returning a `Signal`. The `xpulse` signal is synchronous to `xclk`, whereas the returned signal is synchronous to `clk`.

If the input pulse is longer than a clock period, a `DERIVE` option should be passed in order to receive exactly one output pulse per event. See simulation below.

In [1]:
from cyrite.library.xcd.xclkdomain import XClkDomainSynced
from cyhdl import *

Bool = Signal.Type(bool)

@block
def unit_dc(reset : ResetSignal,
            clka : ClkSignal,
            clkb : ClkSignal,
            in_a  : Bool,
            out_b : Bool.Output
           ):

    x = XClkDomainSynced(reset = reset,
                         xclk = clka,
                         clk = clkb,
                         xpulse = in_a,
                        DERIVE = 'rising')

    wires = [
        out_b    .wireup  (x)
    ]
    return instances()

### Test bench and simulation

The test bench below uses a 1:3 ratio between the clocks for a sane display of the simplified wave trace.

In [2]:
from cyrite.simulation import sim, icarus, ghdl

@sim.testbench(icarus.ICARUS, 'ns')
@block
def tb_dc():
    reset = ResetSignal(False, True, isasync = True)
    a, b = [ Bool(name = n) for n in "ab" ]
    clk_a = ClkSignal(name = 'clka')
    clk_b = ClkSignal(name = 'clkb')

    uut = unit_dc(reset, clk_a, clk_b, a, b)

    @always(delay(6))
    def clkgen_a():
        clk_a.next = ~clk_a

    @always(delay(2))
    def clkgen_b():
        clk_b.next = ~clk_b

    @sequence
    def main():
        yield delay(1)
        a.next = False
        reset.next = True
        yield delay(13)
        reset.next = False

        yield delay(10)

        yield clk_a.negedge
        a.next = True
        yield 5 * (clk_a.negedge, )
        a.next = False

        yield delay(10)
        raise StopSimulation
    return instances()

In [3]:
tb = tb_dc()

/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/kernel/components.py:118: UserWarning: Fallback: Pass through other argument for arg DERIVE (<class 'str'>)
  base.warnings.warn(msg)


Then run the test bench:

In [4]:
tb.run(200, wavetrace = True, debug = True)

 Writing 'flagx' to file /tmp/flagx.v 
DEBUG NAME do <class 'myirl.library.shift.SAlias'>
DEBUG Fallback wire for reset
DEBUG Fallback wire for clka
DEBUG Fallback wire for clkb
 Writing 'unit_dc' to file /tmp/unit_dc.v 
 Writing 'tb_dc' to file /tmp/tb_dc.v 
 Note: Changing library path prefix to /tmp/ 
 Creating library file /tmp/module_defs.v 
DEBUG FILES ['/tmp/flagx.v', '/tmp/unit_dc.v', '/tmp/tb_dc.v']
==== COSIM stdout ====
VCD info: dumpfile tb_dc.vcd opened for output.
Stop Simulation



/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/cyrite/simulation/icarus.py:51: UserWarning: Ignoring wavetrace argument for Verilog simulator
  warnings.warn("Ignoring wavetrace argument for Verilog simulator")


0

### The wave trace

Note we have to take the faster clock of `clka` and `clkb` as sampling clock. Aliasing may occur in the simple wave trace below if the clocks don't have an integer ratio.

In [5]:
from cyrite import waveutils
cfg = { 'tb_dc.clka' : None , 'tb_dc.a' : None, 'tb_dc.clkb' : None,
       'tb_dc.b' : None, 'tb_dc.reset' : None, 
      }
waveutils.draw_wavetrace(tb, 'tb_dc.vcd', 'clkb', cfg = cfg)

## Notes

* `XClkDomainSynced` is a composite class that creates logic and returns a signal instance.
* It can also take `bv` bit vectors as arguments and will operate on each bit
* When using this function to pass register values across clock domains, it is recommended to derive one enable pulse only and cumulate enable and data into one bit vector that is passed through the synchronizer with `DERIVE='none'`.
* The reset pulse must cover both clock periods when synchronous (which is the default).

## Register passing

The `XClkDomainSynced` logic is not restricted to single bit I/O and can also take a bit vector. This could be abused for units measuring pulse durations synchronized to a different clock. When using the RegisterSignal abstraction layer, this can be done as follows:

In [6]:
from cyrite.library.soc import *

We define a register with data and valid bitfields, plus flag it `WRITEONLY | VOLATILE`:

In [7]:
reg01 = Reg(9,
    [
        BF('valid', 8, 8),
        BF("data", 7, 0),
    ], flags = Reg.VOLATILE | Reg.WRITEONLY
)

The cross clock register unit. We pass the `RegisterSignal` through the interface. This actually infers in the HDL as a container with input and output signals. Before creating the test unit, we derive from the RegisterSignal and augment it by a simulation auxiliary method to simulate a register write access:

In [8]:
class MyRegSignal(RegisterSignal):
    
    @cyrite_method.sequence
    def _write(self, clk, en, val):
        yield clk.negedge
        en.next = True
        self.read.valid.next = True
        self.read.data.next = val
        yield clk.negedge
        self.read.valid.next = False
        en.next = False
        
    @cyrite_method.sequence
    def _init(self):
        self.read.data.next = 0x00
        self.read.valid.next = False
        yield delay(1)

## Test unit

The register transfer test unit uses an extra `en` pin to signal when the register was written. This is passed through the synchronizer to the `select.sel_w` signal of the RegisterSignal container.

In [9]:
@block
def xregister(reset : ResetSignal,
              clk_a : ClkSignal,
              clk_b : ClkSignal,
              en    : Bool,
              r : RegisterSignal):
    
    a = Signal(intbv()[9:])

    @always(clk_b)
    def ff():
        # Requires a registered driver for Verilog.
        r.set(u)
        r.select.sel_w.next = v 

    v = XClkDomainSynced(reset = reset,
                     xclk = clk_a,
                     clk = clk_b,
                     xpulse = en)

    u = XClkDomainSynced(reset = reset,
                     xclk = clk_a,
                     clk = clk_b,
                     xpulse = a)
    wires = [ r.assign(a)]

    return instances()


### Testbench

This setup is currently restricted to VHDL output:

In [10]:
@sim.testbench(ghdl.GHDL, 'ns')
@block
def tb_xreg():
    reset = ResetSignal(False, True, isasync = True)
    clk_a = ClkSignal(name = 'clka')
    clk_b = ClkSignal(name = 'clkb')
    en = Bool()
    en_out = Bool()

    rs = MyRegSignal("regsig", template = reg01)
    data_out = Signal(intbv()[9:])

    uut = xregister(reset, clk_a, clk_b, en, rs)

    connections = [
        en_out.wireup(rs.select.sel_w),
        data_out.wireup(rs.write.data)
    ]
    
    @always(delay(6))
    def clkgen_a():
        clk_a.next = ~clk_a

    @always(delay(2))
    def clkgen_b():
        clk_b.next = ~clk_b

    @sequence
    def main():
        yield delay(1)
        en.next = False
        reset.next = True
        yield delay(13)
        reset.next = False

        rs._init()
        
        yield delay(10)

        yield from rs._write(clk_a, en, 0xaa)

        yield delay(30)
        raise StopSimulation
    return instances()

In [11]:
tb = tb_xreg()
tb.run(200, wavetrace = True, debug = False)

 Module tb_xreg: Existing instance flagx, rename to flagx_1 
 Module tb_xreg: Existing instance flagx, rename to flagx_2 
 Writing 'flagx_2' to file /tmp/myirl_tb_xreg_wkgp0hlj/flagx_2.vhdl 
 Writing 'flagx_1' to file /tmp/myirl_tb_xreg_wkgp0hlj/flagx_1.vhdl 
 Writing 'xregister' to file /tmp/myirl_tb_xreg_wkgp0hlj/xregister.vhdl 
 Writing 'tb_xreg' to file /tmp/myirl_tb_xreg_wkgp0hlj/tb_xreg.vhdl 
 Creating library file module_defs.vhdl 


/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/kernel/components.py:118: UserWarning: Fallback: Pass through other argument for arg DERIVE (<class 'str'>)
  base.warnings.warn(msg)


0

## Wave trace

The test bench writes an `0xaa` from the `clka` domain. When the output to the `clkb` domain is valid, the `en_out` is pulsed.

In [12]:
from cyrite import waveutils
cfg = { 'tb_xreg.clka' : None , 'tb_xreg.en' : None, 'tb_xreg.clkb' : None,
       'tb_xreg.inst_xregister_0.a[8:0]' : None, 'tb_xreg.reset' : None, 
       'tb_xreg.en_out' : None, 'tb_xreg.data_out[8:0]' : None
      }
waveutils.draw_wavetrace(tb, 'tb_xreg.vcd', 'clkb', cfg = cfg)

## Design issues

We observe that the data_out signals are repeatedly toggled by the synchronizing pipeline.
This is not pretty and is due to the synchronizer treating each bit as a pulse instead of a static data bit.

A more resource saving approach is, to use the specific `datax` module.

In [13]:
from cyrite.library.xcd.xclkdomain import datax
import inspect
s = inspect.signature(datax.func)
for n, i in s.parameters.items():
    print("'%s'   :    " % n, i.annotation)

'reset'   :     <class 'myirl.kernel._types.ResetSignal'>
'clki'   :     <class 'myirl.kernel._types.ClkSignal'>
'eni'   :     [ Signal Type <class 'myirl.kernel._types._LogicBool'>:1 ]
'di'   :     <class 'myirl.kernel._types.Signal'>
'clko'   :     <class 'myirl.kernel._types.ClkSignal'>
'eno'   :     [ Signal Type <class 'myirl.kernel._types._LogicBool'>:1 ]
'do'   :     <class 'myirl.kernel._types._SigOut'>


Here, the `en*` signals are separate valid bits and the data is passed along internally when no simultaneous clock event takes place.

## Caveats

Due to the internal pipeline length of three samples, a dead time applies to this synchronizer logic. This is minimum three cycles of the slower clock of the two.